<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/dashboarding/dashboarding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install --upgrade google-cloud-bigquery

In [3]:
# This stub (ddfimport) allows the Ddf EE API to be imported.
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_import_common()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 7.36 KiB | 1.84 MiB/s, done.
Resolving deltas: 100% (7/7), done.
executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [4]:
import importlib
import eeddf
import bqddf
importlib.reload(eeddf)
importlib.reload(bqddf)

# Use your credentials to access the BQ tables.
eeddf.initialize_ddf(test_environment = True)

In [5]:
def get_all_training_results():
  client = bqddf._get_big_query_client()

  # Set up SQL query
  table_name = f"{bqddf._CONFIG['DATASET']}.{bqddf._CONFIG['FLATTENED_TABLE']}"
  query = f"SELECT * FROM {table_name}"

  # Execute the query
  results = client.query_and_wait(query)
  return results

results = get_all_training_results()

In [6]:
# Results is iterator based, so you will get an error running this twice!
df = results.to_dataframe()

In [7]:
df.head()

,training_id,num_epochs,num_layers,num_nodes_per_layer,training_batch_size,learning_rate,dropout_rate,activation_func,early_stopping_patience,double_sided_kl,...,mean_rmse,var_rmse,overall_rmse,dataset_id,completion_timestamp,radius_pace,per_radius_eval,tags,as_json,trusted_buffer_radius
0,test-rkrige-2025-01-31-2,-1,-1,-1,-1,-1.00000,-1.0,,-1,False,...,1.923385,1.253441,1.588413,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-03-14 16:22:24.897040+00:00,100,"[{'radius': 6, 'auc': 0.5499226375117593, 'p_v...","[author:npr, regression_kriging, all_standardi...","{""activation_func"":"""",""double_sided_kl"":false,...",5
1,test-rkrige-2025-01-31-2,-1,-1,-1,-1,-1.00000,-1.0,,-1,False,...,1.923385,1.253441,1.588413,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-03-14 16:43:15.137370+00:00,100,"[{'radius': 6, 'auc': 0.5306714602171038, 'p_v...","[author:npr, regression_kriging, all_standardi...","{""activation_func"":"""",""double_sided_kl"":false,...",5
2,cv3-test-4,5000,2,20,8,0.00001,0.0,relu,100,False,...,1.475324,1.074420,1.274872,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-04-04 20:09:14.656231+00:00,100,"[{'radius': 6, 'auc': 0.46058722929579676, 'p_...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5
3,cv3-test-3,5000,2,20,8,0.00001,0.0,relu,100,False,...,1.475324,1.074420,1.274872,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-04-04 19:51:43.483638+00:00,100,"[{'radius': 6, 'auc': 0.46058722929579676, 'p_...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5
4,test-2025-01-18-cv2-test-erickzul,1000,2,20,8,0.00010,0.0,relu,100,False,...,1.446195,2.116459,1.781327,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-28 16:16:47.436258+00:00,100,"[{'radius': 6, 'auc': 0.4367088058965503, 'p_v...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5


In [27]:
# @title Experiments by a particular author sorted by timestamp

AUTHOR = 'ruru'  #@param {type: "string"}
AUTHOR_PREFIX = "author:"

In [28]:
# Function to safely extract author name
def get_author_if_valid(tags_list):
  if len(tags_list) > 0:
    first_tag = tags_list[0]
    if AUTHOR_PREFIX in first_tag:
      potential_author = first_tag.split(':', 1)[1]
      if potential_author:
        return potential_author
  return None

In [29]:
df['extracted_author'] = df['tags'].apply(get_author_if_valid)

In [31]:
specific_author_df = df[df['extracted_author'] == AUTHOR].copy() # .copy() to avoid SettingWithCopyWarning

In [33]:
specific_author_df.sort_values(by='completion_timestamp', inplace=True)

In [34]:
specific_author_df

,training_id,num_epochs,num_layers,num_nodes_per_layer,training_batch_size,learning_rate,dropout_rate,activation_func,early_stopping_patience,double_sided_kl,...,var_rmse,overall_rmse,dataset_id,completion_timestamp,radius_pace,per_radius_eval,tags,as_json,trusted_buffer_radius,extracted_author
120,test-2025-01-18-cv2-test,1000,2,20,8,0.00010,0.0,relu,100,False,...,1.130919,1.631735,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-01-24 17:40:50.315733+00:00,100,"[{'radius': 6, 'auc': 0.529490162594798, 'p_va...","[author:ruru, ViNN, test, new_tag, new_tag, ne...","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
118,test-2025-01-18-cv2-test,1000,2,20,8,0.00010,0.0,relu,100,False,...,1.130919,1.631735,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-01-24 17:44:14.831316+00:00,100,"[{'radius': 6, 'auc': 0.5065504122944507, 'p_v...","[author:ruru, ViNN, test, new_tag, new_tag, ne...","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
121,ruru-w-carbon,2000,2,20,8,0.00100,0.0,relu,200,False,...,6.362523,4.663733,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-14 16:59:48.416696+00:00,100,"[{'radius': 6, 'auc': 0.5237419393717669, 'p_v...","[author:ruru, ViNN, Carbon-test]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
124,ruru-w-carbon-3layers,2000,3,12,3,0.00050,0.0,relu,200,False,...,2.063757,2.557515,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-14 17:03:33.548258+00:00,100,"[{'radius': 6, 'auc': 0.5149439751923366, 'p_v...","[author:ruru, ViNN, Carbon-test]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
122,ruru-w-carbon-2layers,2000,2,10,12,0.00050,0.0,relu,200,False,...,2.278469,2.191581,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-14 17:46:15.520978+00:00,100,"[{'radius': 6, 'auc': 0.49487456388701695, 'p_...","[author:ruru, ViNN, Carbon-test]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
123,ruru-w-carbon-2layers-1b,2000,2,20,1,0.00050,0.0,relu,200,False,...,4.375263,2.755721,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-14 18:05:52.325815+00:00,100,"[{'radius': 6, 'auc': 0.4906411670633953, 'p_v...","[author:ruru, ViNN, Carbon-test]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
4,test-2025-01-18-cv2-test-erickzul,1000,2,20,8,0.00010,0.0,relu,100,False,...,2.116459,1.781327,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-28 16:16:47.436258+00:00,100,"[{'radius': 6, 'auc': 0.4367088058965503, 'p_v...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
5,test-2025-01-18-cv2-test-erickzul-2,1000,2,20,8,0.00010,0.0,relu,100,False,...,1.203989,1.613528,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-02-28 19:22:14.901177+00:00,100,"[{'radius': 6, 'auc': 0.5075010366767363, 'p_v...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
6,cv3-test,1000,2,20,8,0.00010,0.0,relu,100,False,...,1.859029,1.708542,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-04-04 15:50:45.135274+00:00,100,"[{'radius': 6, 'auc': 0.4904906624111679, 'p_v...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru
3,cv3-test-3,5000,2,20,8,0.00001,0.0,relu,100,False,...,1.074420,1.274872,/content/gdrive/MyDrive/amazon_rainforest_file...,2025-04-04 19:51:43.483638+00:00,100,"[{'radius': 6, 'auc': 0.46058722929579676, 'p_...","[author:ruru, ViNN]","{""activation_func"":""relu"",""double_sided_kl"":fa...",5,ruru


In [35]:
# @title Experiments by a particular author sorted by RMSE
specific_author_df.sort_values(by='overall_rmse', inplace=True, ascending=False)

In [44]:
for _,row in specific_author_df.iterrows():
  print((row['per_radius_eval']))
  break

[{'radius': 6, 'auc': 0.5355438316354266, 'p_value': 3.552713678800501e-15, 'precision_target': 0.5384615384615384, 'recall_target': 0.3181818181818182, 'pr_curve': array([{'precision': array([0.5       , 0.51162791, 0.5       , 0.48780488, 0.5       ,
               0.51282051, 0.5       , 0.48648649, 0.5       , 0.48571429,
               0.5       , 0.48484848, 0.5       , 0.48387097, 0.5       ,
               0.51724138, 0.5       , 0.48148148, 0.5       , 0.48      ,
               0.5       , 0.52173913, 0.5       , 0.52380952, 0.5       ,
               0.47368421, 0.44444444, 0.41176471, 0.4375    , 0.46666667,
               0.5       , 0.53846154, 0.5       , 0.4       , 0.44444444,
               0.5       , 1.        ]), 'recall': array([1.        , 1.        , 0.95454545, 0.90909091, 0.90909091,
               0.90909091, 0.86363636, 0.81818182, 0.81818182, 0.77272727,
               0.77272727, 0.72727273, 0.72727273, 0.68181818, 0.68181818,
               0.68181818, 0.